In [1]:
import requests
from bs4 import BeautifulSoup
import time
import re
import pandas as pd
import csv

In [2]:
def get_soup(url):
    while True:
        try:
            res = requests.get(url)
            break
        except:
            pass
    html = res.content
    soup = BeautifulSoup(html,'html.parser')
    return soup

In [3]:
def name_number(url, select_name, region_num=None):
    if region_num:
        link=url+'az/smenu'+region_num+'.htm'
        soup=get_soup(link)
        
    else:
        soup=get_soup(url)

    select=soup.find('select', {'name':select_name})

    option=select.find_all('option')

    number=[i.get('value') for i in option][1:]


    name=[i.strip('\r') for i in select.text.split('\n') if i.strip('\r')!=''][1:]
    
    name_number={number[i]: name[i] for i in range(len(number))}
    
    return name_number

In [4]:
print(name_number("http://mekteb.edu.az/", 'region'))

{'12': 'Bakı ş. Binəqədi rayonu', '14': 'Bakı ş. Xətai rayonu', '15': 'Bakı ş. Xəzər rayonu', '20': 'Bakı ş. Qaradağ rayonu', '16': 'Bakı ş. Nərimanov rayonu', '19': 'Bakı ş. Nəsimi rayonu', '21': 'Bakı ş. Nizami rayonu', '22': 'Bakı ş. Pirallahı rayonu', '18': 'Bakı ş. Sabunçu rayonu', '13': 'Bakı ş. Səbayel rayonu', '17': 'Bakı ş. Suraxanı rayonu', '11': 'Bakı ş. Yasamal rayonu', '31': 'Naxçıvan şəhəri', '32': 'Naxçıvan MR Babək ray.', '35': 'Naxçıvan MR Culfa ray.', '30': 'Naxçıvan MR Kəngərli ray.', '33': 'Naxçıvan MR Ordubad ray.', '34': 'Naxçıvan MR Sədərək ray.', '37': 'Naxçıvan MR Şahbuz ray.', '36': 'Naxçıvan MR Şərur ray.', '48': 'Abşeron rayonu', '65': 'Ağcabədi rayonu', '66': 'Ağdam rayonu', '72': 'Ağdaş rayonu', '45': 'Ağstafa rayonu', '74': 'Ağsu rayonu', '90': 'Astara rayonu', '59': 'Balakən rayonu', '79': 'Beyləqan rayonu', '63': 'Bərdə rayonu', '85': 'Biləsuvar rayonu', '76': 'Cəbrayıl rayonu', '93': 'Cəlilabad rayonu', '39': 'Daşkəsən rayonu', '78': 'Füzuli rayonu', '

In [5]:
print(name_number("http://mekteb.edu.az/", 'school', '38'))

{'38001': '1 saylı şəhər orta məktəbi -İ.Qayıbov adına', '38002': '2 saylı şəhər orta məktəbi -Ə.Cavad adına', '38003': '3 saylı şəhər orta məktəbi', '38004': '4 saylı şəhər orta məktəbi -M.Mehdizadə adına', '38005': '5 saylı şəhər orta məktəbi -M.Ə.Sabir adına', '38006': '6 saylı şəhər orta məktəbi -C.Cabbarlı adına', '38007': '7 saylı şəhər orta məktəbi', '38008': '8 saylı şəhər orta məktəbi -M.P.Vaqif adına', '38009': '9 saylı şəhər orta məktəbi -A.S.Puşkin adına', '38010': '10 saylı şəhər orta məktəbi -M.Hadi adına', '38011': '11 saylı şəhər orta məktəbi -Nizami adına', '38012': '12 saylı şəhər orta məktəbi -V.Əliyev adına', '38013': '13 saylı şəhər orta məktəbi -Ş.İ.Xətai adına', '38014': '14 saylı şəhər orta məktəbi -M.Müşfiq adına', '38015': '15 saylı şəhər orta məktəbi -M.Əzizbəyov adına', '38016': '16 saylı şəhər orta məktəbi -M.Ş.Vazeh adına', '38017': '17 saylı şəhər orta məktəbi', '38018': '18 saylı şəhər orta məktəbi -Nizami Aydın adına', '38019': '19 saylı şəhər orta məkt

In [6]:
def qabaqcil_mezunlardaki_urls(region_nomre, mekteb_nomre):
    url=f'http://mekteb.edu.az/az/{region_nomre}/{mekteb_nomre}a0000_3a.htm'
    
    base_url=f'http://mekteb.edu.az/az/{region_nomre}/'
    
    soup=get_soup(f'http://mekteb.edu.az/az/{region_nomre}/{mekteb_nomre}a0000_3a.htm')
    
    alar=soup.find_all('a')
    
    urls=[base_url+i.get('href') for i in alar if '_3' in i.get('href')]
    
    urls.append(url)
    
    return urls

In [7]:
print(qabaqcil_mezunlardaki_urls('38', '38031'))

['http://mekteb.edu.az/az/38/38031a0000_3b.htm', 'http://mekteb.edu.az/az/38/38031a0000_3c.htm', 'http://mekteb.edu.az/az/38/38031a0000_3d.htm', 'http://mekteb.edu.az/az/38/38031a0000_3e.htm', 'http://mekteb.edu.az/az/38/38031a0000_3a.htm']


In [8]:
def qabaqcil_mezunlar(url):
    soup=get_soup(url)
    td=soup.find_all('td', {'bgcolor':'#FCFCE7'})
    for i in td:
        melumatlar=i.text.strip().replace('\r', '').split('\n')
        if len(melumatlar)!=4:
            ad_soyad, il, bal=melumatlar
            il=re.findall('\d{4}', il)[0]
            bal=re.findall('\d{3}', bal)[0]
            uni='Universitet bilinmir'
            ixtisas='Ixtisas bilinmir'
        else:
            ad_soyad, il, bal, uni_ixtisas=melumatlar

            il=re.findall('\d{4}', il)[0]
            bal=re.findall('\d{3}', bal)[0]
            uni_ixtisas=uni_ixtisas.strip(' ')

            if uni_ixtisas[0]=='"':
                uni_ixtisas=uni_ixtisas.strip('"').replace('"', '', 1)

            uni=uni_ixtisas[: uni_ixtisas.index('"')].strip()[:-3]  
            ixtisas=uni_ixtisas[uni_ixtisas.index('"')+1:uni_ixtisas.rfind('"')]
      
        yield(ad_soyad.strip(), uni, ixtisas, il, bal)

In [9]:
region_number=name_number('http://mekteb.edu.az/', 'region')

In [10]:
region_number

{'12': 'Bakı ş. Binəqədi rayonu',
 '14': 'Bakı ş. Xətai rayonu',
 '15': 'Bakı ş. Xəzər rayonu',
 '20': 'Bakı ş. Qaradağ rayonu',
 '16': 'Bakı ş. Nərimanov rayonu',
 '19': 'Bakı ş. Nəsimi rayonu',
 '21': 'Bakı ş. Nizami rayonu',
 '22': 'Bakı ş. Pirallahı rayonu',
 '18': 'Bakı ş. Sabunçu rayonu',
 '13': 'Bakı ş. Səbayel rayonu',
 '17': 'Bakı ş. Suraxanı rayonu',
 '11': 'Bakı ş. Yasamal rayonu',
 '31': 'Naxçıvan şəhəri',
 '32': 'Naxçıvan MR Babək ray.',
 '35': 'Naxçıvan MR Culfa ray.',
 '30': 'Naxçıvan MR Kəngərli ray.',
 '33': 'Naxçıvan MR Ordubad ray.',
 '34': 'Naxçıvan MR Sədərək ray.',
 '37': 'Naxçıvan MR Şahbuz ray.',
 '36': 'Naxçıvan MR Şərur ray.',
 '48': 'Abşeron rayonu',
 '65': 'Ağcabədi rayonu',
 '66': 'Ağdam rayonu',
 '72': 'Ağdaş rayonu',
 '45': 'Ağstafa rayonu',
 '74': 'Ağsu rayonu',
 '90': 'Astara rayonu',
 '59': 'Balakən rayonu',
 '79': 'Beyləqan rayonu',
 '63': 'Bərdə rayonu',
 '85': 'Biləsuvar rayonu',
 '76': 'Cəbrayıl rayonu',
 '93': 'Cəlilabad rayonu',
 '39': 'Daşkəsən 

In [11]:
region_schools={}
for r_nomre in list(region_number.keys()):
    res=name_number('http://mekteb.edu.az/', 'school', r_nomre)
    region_schools[r_nomre]=res

In [12]:
region_schools

{'12': {'12002': '2 N-li orta məktəb',
  '12003': '3 saylı orta məktəb',
  '12004': '4 N-li orta məktəb',
  '12005': '5 N-li orta məktəb',
  '12006': '6 N-li orta məktəb',
  '12029': '115a saylı xüsusi əyani-qiyabi orta məktəb (cəzaçəkmə müəssisəsi nəzdində)',
  '12030': '30 saylı orta məktəb',
  '12083': '83 saylı məktəb-lisey (əvvəlki 83 saylı orta məktəb )',
  '12099': '99 saylı orta məktəb',
  '12100': '100 saylı orta məktəb',
  '12102': '102 saylı orta məktəb',
  '12103': '103 saylı orta məktəb',
  '12115': '115 saylı orta məktəb',
  '12126': '126 saylı orta məktəb',
  '12135': '135 saylı orta məktəb',
  '12143': '143 saylı orta məktəb',
  '12144': '144 saylı orta məktəb',
  '12157': '157 saylı orta məktəb',
  '12179': '179 saylı orta məktəb',
  '12182': '182 saylı orta məktəb',
  '12205': '205 saylı orta məktəb',
  '12217': '217 saylı orta məktəb',
  '12244': '244 saylı orta məktəb',
  '12246': '246 saylı orta məktəb',
  '12248': '248 saylı orta məktəb',
  '12249': '248 saylı ort

In [13]:
data_file=open('data.csv', 'w', encoding='utf-8')

In [14]:
for s_k in region_schools.keys():
    for m_k in region_schools[s_k]:
        linkler=qabaqcil_mezunlardaki_urls(s_k, m_k)
        for link in linkler:
            for mezun in qabaqcil_mezunlar(link):
                if mezun:
                    data_file.write(';'.join(mezun)+';'+region_number[s_k]+';'+s_k+';'+region_schools[s_k][m_k]+';'+m_k+"\n")
data_file.close()

In [15]:
df=pd.read_csv('data.csv', sep=';', header=None,
              names=['ad soyad', 'universitet', 'ixtisas', 'buraxilis ili', 'bal', 'region', 'rayon nomre', 'mekteb', 'mekteb nomre'])

In [16]:
df.head()

,ad soyad,universitet,ixtisas,buraxilis ili,bal,region,rayon nomre,mekteb,mekteb nomre
0,Süleymanlı Turqut Emin,Universitet bilinmir,Ixtisas bilinmir,2019,620,Bakı ş. Binəqədi rayonu,12,3 saylı orta məktəb,12003
1,Hüseynzadə Aynur Mirrasim,Azərbaycan Dövlət Neft və Sənaye Universiteti,Biznesin idarə edilməsi (tədris ingilis dilində),2019,601,Bakı ş. Binəqədi rayonu,12,3 saylı orta məktəb,12003
2,Cavadlı Ceyhun Vüsal,Bakı Ali Neft Məktəbi,Neft- qaz mühəndisliyi (tədris ingilis dilində),2019,595,Bakı ş. Binəqədi rayonu,12,3 saylı orta məktəb,12003
3,Ələskərova Nəzrin Elşad,ADA Universiteti,Hüquqşünaslıq (tədris ingilis dilində),2019,589,Bakı ş. Binəqədi rayonu,12,3 saylı orta məktəb,12003
4,Xəlilov Emin Daşqın,Azərbaycan Dövlət İqtisad Universiteti,Mühasibat uçotu və audit,2019,582,Bakı ş. Binəqədi rayonu,12,3 saylı orta məktəb,12003
